# Export Database

This notebook outlines the steps to export data from a DuckDB database:

1. Initialize the `DataExporter` class.
2. Export data for all ZIP codes and industry levels.
3. Verify the export results.

The `DataExporter` class will manage the data extraction and export operations.

**Important:** Only run this notebook if you do not already have the CSV files.  
Otherwise, this notebook should be used for reference only.  
The cell outputs are provided for your reference to understand the expected behavior.


In [ ]:
import os
import logging
import duckdb
from multiprocessing import Pool
from tqdm import tqdm
import query as q
from duck_db_exporter import DataExporter  # Assuming DataExporter is defined in data_exporter.py

# Configure logging
logging.basicConfig(level=logging.INFO)

## I. Export Database
- this is in the jupyterversion, there will also be a script version that be used by github action.

### I.(a) Export a Single Annual Data

In [2]:
# Initialize DataExporter
exporter = DataExporter(
    base_db_path='../zip_data/duck_db_manager/database/',
    threads=4,
    export_dir='../../US/zip',
    industry_levels=[2, 5, 6],  # Specify industry levels if needed
    year=2019  # Specify the year for the data
)

In [3]:
file_paths = exporter.make_csv()  # No state specified: exports data for all states. You can also specify a specific state to export.

INFO:root:Export for state OK at industry level 2 is finished. 3951 rows have been exported.
INFO:root:Export for state OK at industry level 5 is finished. 8408 rows have been exported.
INFO:root:Export for state OK at industry level 6 is finished. 8482 rows have been exported.
INFO:root:Export for state NC at industry level 2 is finished. 7120 rows have been exported.
INFO:root:Export for state NC at industry level 5 is finished. 21167 rows have been exported.
INFO:root:Export for state NC at industry level 6 is finished. 21735 rows have been exported.
INFO:root:Export for state KS at industry level 2 is finished. 3589 rows have been exported.
INFO:root:Export for state KS at industry level 5 is finished. 6630 rows have been exported.
INFO:root:Export for state KS at industry level 6 is finished. 6616 rows have been exported.
INFO:root:Export for state MN at industry level 2 is finished. 5848 rows have been exported.
INFO:root:Export for state MN at industry level 5 is finished. 13905

### I.(b) Export All Annual Data in a Single Run

In [4]:
export_results = {}
for year in range(2012, 2023):
    # Initialize DataExporter
    exporter = DataExporter(
        base_db_path='../zip_data/duck_db_manager/database/',
        threads=4,
        export_dir='../../US/zip',
        industry_levels=[2, 5, 6],  # Specify industry levels if needed
        year=year  # Specify the year for the data
    )
    # Export data for all states
    export_results[year] = exporter.make_csv()

INFO:root:Export for state RI at industry level 2 is finished. 1072 rows have been exported.
INFO:root:Export for state RI at industry level 5 is finished. 9405 rows have been exported.
INFO:root:Export for state RI at industry level 6 is finished. 10413 rows have been exported.
INFO:root:Export for state SD at industry level 2 is finished. 2866 rows have been exported.
INFO:root:Export for state SD at industry level 5 is finished. 11782 rows have been exported.
INFO:root:Export for state SD at industry level 6 is finished. 12593 rows have been exported.
INFO:root:Export for state CA at industry level 2 is finished. 25624 rows have been exported.
INFO:root:Export for state CA at industry level 5 is finished. 225341 rows have been exported.
INFO:root:Export for state CA at industry level 6 is finished. 254130 rows have been exported.
INFO:root:Export for state MA at industry level 2 is finished. 7653 rows have been exported.
INFO:root:Export for state MA at industry level 5 is finished.

In [4]:
def display_export_summary(export_results):
    """
    Displays the summary of export results for different years.

    Args:
        export_results (dict): A dictionary where keys are years and values are tuples of (industry_level_row_counts, total_rows_exported).
                               `industry_level_row_counts` is a dict with industry levels as keys and row counts as values.
                               `total_rows_exported` is the total number of rows exported for that year.
    """
    print("Export Summary:")
    print("-" * 50)
    
    for year, (industry_level_row_counts, total_rows_exported) in export_results.items():
        print(f"Year: {year}")
        for level, count in industry_level_row_counts.items():
            print(f"  Industry Level {level}: {count} rows")
        print(f"  Total Rows: {total_rows_exported}")
        print("-" * 50)

In [ ]:
display_export_summary(export_results)

## II. Tests After Export

In this section, we will validate the consistency of the exported CSV files against the original DuckDB database. These tests will ensure that the exported files contain the expected data and accurately reflect the data from the database.

The following tests will be performed:
1. Unit tests to ensure the functionality of the `DataExporter` is robust.
2. Validation to confirm that the data in the CSV files matches the data in the DuckDB database.

### II.(a) Unit Test for Data Exporter Accuracy

This section validates the accuracy of data exported by the `DataExporter` class. It ensures that the exported CSV files correctly reflect the data in the DuckDB database.

#### Description

The notebook performs the following tasks:
- **Setup**: Creates a temporary DuckDB database with test data.
- **Export**: Uses the `DataExporter` to generate CSV files.
- **Validation**: 
  - Checks that the number of rows in the CSV files matches the database.
  - Verifies that the data in the CSV files is consistent with the database.
- **Teardown**: Removes test files and the temporary database.

#### Key Tests

- **Row Count Check**: Ensures that the number of rows in the exported CSV files matches those in the database.
- **Data Consistency Check**: Confirms that the data values and types in the CSV files align with the database.

#### Usage

Execute the cells in the notebook to perform the tests. The notebook will automatically set up the test environment, run the validations, and clean up afterward.


In [ ]:
# Execute the test script
%run data_exporter_unit_test.py

### II.(b) Data Export Validation Test

This test ensures that the CSV files exported from our DuckDB database match the data stored in the database.

**Description:**
The `DataExporterTest` class validates the accuracy of exported CSV files by comparing them with the data in the DuckDB database. This process checks for consistency across all states and industry levels, ensuring that the exported files correctly represent the database records.

**Key Tests:**
1. **Initialization:** Configure the `DataExporterTest` with the target year and directory paths.
2. **Data Fetching:** Retrieve data from the database for all states and industry levels.
3. **CSV Comparison:** Compare the contents of each CSV file with the corresponding database data.
4. **Mismatch Reporting:** Log any discrepancies with detailed information, including file paths and data previews.

**Usage:**
To run the test:
1. Initialize the `DataExporterTest` with the desired year and directory paths.
2. Execute the test script.
3. Review the logs for any discrepancies between the CSV files and the database data.

The test will help ensure the integrity and accuracy of the exported data files.


In [3]:
from data_exporter_validation_test import DataExporterTest

#### Validate a Single Year：

In [4]:
# Replace '2019' with your desired year
# The default values for the Constructor's arguments are: 
# base_db_path='../zip_data/duck_db_manager/database/', export_dir='../../US/zip'
# Update these paths if any changes occur:
tester = DataExporterTest(year=2019) 
tester.run_test()

All CSV files match the database data by row count.


#### Validate for All Years:

In [ ]:
for year in range(2012, 2023):
    tester = DataExporterTest(year=year)
    tester.run_test()

Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
Tested Year {year}: All CSV files match the database data by row count.
